In [ ]:
RANDOM_STATE = 15012024

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostRegressor

import random
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
db_config = {
'user': 'praktikum_student',
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432,
'db': 'data-science-final'
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(RANDOM_STATE)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
query = """
             SELECT * FROM steel.data_bulk
        """
data_bulk = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_arc
        """
data_arc = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_bulk_time
        """
data_bulk_time = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_gas
        """
data_gas = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_temp
        """
data_temp = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_wire
        """
data_wire = pd.io.sql.read_sql(query, con=engine)

query = """
             SELECT * FROM steel.data_wire_time
        """
data_wire_time = pd.io.sql.read_sql(query, con=engine)

In [ ]:
data_arc

In [ ]:
data_arc.info()

In [ ]:
data_arc["BeginHeat"] = data_arc["BeginHeat"].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
data_arc["EndHeat"] = data_arc["EndHeat"].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
data_arc["TotalSeconds"] = data_arc["EndHeat"] - data_arc["BeginHeat"]
data_arc = data_arc.drop(["BeginHeat", "EndHeat"], axis=1)

In [ ]:
plt.boxplot(data_arc["TotalSeconds"])
plt.ylabel("Секунды")
plt.title("Значения в секундах")
plt.show()

In [ ]:
values_to_exclude = data_arc.loc[(data_arc.TotalSeconds < 0) | (data_arc.TotalSeconds > 295), "key"].unique()
data_arc = data_arc[~data_arc["key"].isin(values_to_exclude)].reset_index(drop=True)

In [ ]:
plt.boxplot(data_arc.TotalSeconds)
plt.ylabel("Секунды")
plt.title("Значения в секундах")
plt.show()

In [ ]:
plt.boxplot(data_arc.ActivePower)
plt.title("Значения активной мощности")
plt.ylabel("Мощность")
plt.show()

In [ ]:
values_to_exclude = data_arc.loc[data_arc.ActivePower > 1.25, "key"].unique()
data_arc = data_arc[~data_arc["key"].isin(values_to_exclude)].reset_index(drop=True)
plt.boxplot(data_arc.ActivePower)
plt.title("Значения активной мощности")
plt.ylabel("Мощность")
plt.show()

In [ ]:
plt.boxplot(data_arc.ReactivePower)
plt.title("Значения реактивной мощности")
plt.ylabel("Мощность")
plt.show()

In [ ]:
values_to_exclude = data_arc.loc[(data_arc.ReactivePower < 0) | (data_arc.ReactivePower > 0.95), "key"].unique()
data_arc = data_arc[~data_arc["key"].isin(values_to_exclude)].reset_index(drop=True)
plt.boxplot(data_arc.ReactivePower)
plt.title("Значения реактивной мощности")

In [ ]:
data_arc = data_arc.reset_index(drop=True)
data_arc["PowerFactor"] = data_arc.ActivePower / (data_arc.ActivePower + (-1 * data_arc.ReactivePower))
data_arc

In [ ]:
data_arc = data_arc.groupby("key").median()
data_arc

In [ ]:
data_bulk

In [ ]:
data_bulk.info()

In [ ]:
data_bulk = data_bulk.fillna(0)

In [ ]:
data_bulk_time

In [ ]:
data_bulk_time.info()

In [ ]:
for column in data_bulk_time.drop("key", axis=1).columns:
    data_bulk_time[column] = data_bulk_time[column].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second if x is not None else None)

In [ ]:
a = "Bulk 15"
for column in data_bulk_time.drop("key", axis=1).columns:
     data_bulk_time[column] = data_bulk_time[a] - data_bulk_time[column]
     data_bulk_time[column] = data_bulk_time[column].apply(lambda x: x if x >= 0 else -x)
data_bulk_time = data_bulk_time.fillna(0)

In [ ]:
for column in data_bulk_time.drop("key", axis=1).columns:
     print(f"{column}: {data_bulk_time[column].max()}")
data_bulk_time = data_bulk_time.drop([a, "Bulk 2", "Bulk 8"], axis=1)

In [ ]:
data_gas

In [ ]:
data_gas.info()

In [ ]:
plt.boxplot(data_gas["gas"])
plt.title("Газ")
plt.ylabel("Объем")
plt.show()
data_gas = data_gas.loc[data_gas["gas"] < 20]

In [ ]:
plt.boxplot(data_gas["gas"])
plt.title("Газ")
plt.ylabel("Объем")
plt.show()

In [ ]:
data_temp

In [ ]:
data_temp.info()

In [ ]:
values_to_exclude = data_temp.loc[data_temp["Temperature"].isna() == True, "key"].unique()
data_temp = data_temp[~data_temp["key"].isin(values_to_exclude)].reset_index(drop=True)

In [ ]:
plt.boxplot(data_temp["Temperature"])
plt.title("Температура")
plt.ylabel("C")
plt.show()

In [ ]:
data_temp = data_temp.loc[data_temp["Temperature"] > 1500]
values_to_exclude = data_temp.loc[(data_temp["Temperature"] < 1560) | (data_temp["Temperature"] > 1619), "key"].unique()
data_temp = data_temp[~data_temp["key"].isin(values_to_exclude)].reset_index(drop=True)
plt.boxplot(data_temp["Temperature"])
plt.title("Температура")
plt.ylabel("C")
plt.show()

In [ ]:
key_counts = data_temp['key'].value_counts()
unique_keys = key_counts[key_counts < 2].index.tolist()
data_temp = data_temp[~data_temp['key'].isin(unique_keys)]

In [ ]:
data_temp = data_temp.drop_duplicates(subset="key", keep="last").drop("MesaureTime", axis=1).reset_index(drop = True)
data_temp

In [ ]:
data_wire

In [ ]:
data_wire.info()

In [ ]:
data_wire = data_wire.fillna(0)

In [ ]:
data_wire_time

In [ ]:
data_wire_time.info()

In [ ]:
for column in data_wire_time.drop("key", axis=1).columns:
    data_wire_time[column] = data_wire_time[column].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second if x is not None else None)

In [ ]:
a = "Wire 9"
for column in data_wire_time.drop("key", axis=1).columns:
     data_wire_time[column] = data_wire_time[a] - data_wire_time[column]
     data_wire_time[column] = data_wire_time[column].apply(lambda x: x if x >= 0 else -x)
data_wire_time = data_wire_time.fillna(0)
data_bulk_time = data_wire_time.drop(a, axis=1)

In [ ]:
for column in data_wire_time.drop("key", axis=1).columns:
     print(f"{column}: {data_wire_time[column].max()}")
data_wire_time = data_wire_time[["key", "Wire 1", "Wire 1"]]

In [ ]:
data = pd.merge(data_arc, data_bulk, on='key', how='outer')
data = pd.merge(data, data_gas, on='key', how='outer')
data = pd.merge(data, data_temp, on='key', how='outer')
data = pd.merge(data, data_wire, on='key', how='outer')
data = data.drop("key", axis=1).dropna(axis=0).reset_index(drop=True)

In [ ]:
data

In [ ]:
data.drop("Temperature", axis=1).hist(figsize=(20, 20), bins=100)
plt.suptitle('Гистограммы распределения признаков', x=0.5, y=0.92, fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(data.corr(), cmap='coolwarm', center=0, annot=True, linewidths=2, linecolor='black');

In [ ]:
drop_columns = ["Bulk 1", "Bulk 2", "Bulk 5", "Bulk 6", "Bulk 7", "Bulk 8",
                "Bulk 9", "Bulk 10", "Bulk 11", "Bulk 13", "Wire 2",
                "Wire 3", "Wire 4", "Wire 5", "Wire 6", "Wire 7", "Wire 8", "Wire 9", 
                "ActivePower", "ReactivePower",]
data = data.drop(drop_columns, axis=1)

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(data.corr(), cmap='coolwarm', center=0, annot=True, linewidths=2, linecolor='black');

In [ ]:
data.drop("Temperature", axis=1).hist(figsize=(20, 20), bins=100)
plt.suptitle('Гистограммы распределения признаков', x=0.5, y=0.92, fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(data.corr(), cmap='coolwarm', center=0, annot=True, linewidths=2, linecolor='black');

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.25, random_state=RANDOM_STATE)
data_train = data_train.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)

In [ ]:
results = pd.DataFrame(columns=["Name", "MAE", "R2", "RMSE"])
def add_table(data:(str, float, float, float)):
    a = results.shape[0]
    results.loc[a] = data
    display(results)

In [ ]:
target_column = "Temperature"

In [ ]:
columns = data_train.drop(target_column, axis=1).columns
scaler = StandardScaler()
data_train[columns] = scaler.fit_transform(data_train.drop(target_column, axis=1))
data_test[columns] = scaler.transform(data_test.drop(target_column, axis=1))

In [ ]:
param_grid = {"strategy":["mean"]}
model = DummyRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=make_scorer(mean_absolute_error))

In [ ]:
grid_search.fit(data_train.drop(target_column, axis=1), data_train[target_column])
dummy_model = grid_search.best_estimator_
grid_search.best_score_

In [ ]:
param_grid = {"n_jobs":[6]}
model = LinearRegression()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=make_scorer(mean_absolute_error))

In [ ]:
grid_search.fit(data_train.drop(target_column, axis=1), data_train[target_column])
liniar_model = grid_search.best_estimator_
grid_search.best_score_

In [ ]:
param_grid = {"n_jobs":[6], 
              "max_depth":[3, 4, 5, 6, 7, 8, 9, 10],
              "n_estimators":[100, 200]}
model = RandomForestRegressor(random_state=RANDOM_STATE)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=make_scorer(mean_absolute_error))
grid_search.fit(data_train.drop(target_column, axis=1), data_train[target_column])
forest_model = grid_search.best_estimator_
grid_search.best_score_

In [ ]:
param_grid = {"depth": [3, 5],
              "iterations": [1000, 2000]}
model = model = CatBoostRegressor(random_state=RANDOM_STATE, silent=True)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=make_scorer(mean_absolute_error), n_jobs=8)
grid_search.fit(data_train.drop(target_column, axis=1), data_train[target_column])
catboost_model = grid_search.best_estimator_
grid_search.best_score_

In [ ]:
class CustomDataset(Dataset):
     def __init__(self, data:pd.DataFrame, train=True):
          self.train = train
          if self.train:
             self.data = data.drop(target_column, axis=1)
             self.target = data[target_column]
          else:
             self.data = data.drop(target_column, axis=1)
     
     def __len__(self):
         return len(self.data)
         
     def __getitem__(self, index):
          if self.train:
             data = self.data.iloc[index]
             target = self.target.iloc[index]

             with torch.no_grad():
                 data = torch.tensor(data, dtype=torch.float32, device=DEVICE)
                 target = torch.tensor(target, dtype=torch.float32, device=DEVICE)
                 return data, target
             
          else:
              data = self.data.iloc[index]

              with torch.no_grad():
                 data = torch.tensor(data, dtype=torch.float32, device=DEVICE)
                 return data

In [ ]:
data_train_dataset = CustomDataset(data_train)
data_train_dataloader = DataLoader(data_train_dataset, batch_size=500, shuffle=True)

In [ ]:
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
class Net(nn.Module):
    def __init__(self, input_features, depth):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(input_features, input_features * depth)
        self.fc2 = nn.Linear(input_features * depth, input_features * depth)
        self.fc3 = nn.Linear(input_features * depth, input_features * depth)
        self.fc4 = nn.Linear(input_features * depth, input_features * depth)
        self.fc5 = nn.Linear(input_features * depth, input_features * depth)
        self.fc6 = nn.Linear(input_features * depth, 1)
        self.relu = nn.ReLU()
        self.rrelu = nn.LeakyReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.rrelu(x)
        x = self.fc2(x)
        x = self.rrelu(x)
        x = self.fc3(x)
        x = self.rrelu(x)
        x = self.fc4(x)
        x = self.rrelu(x)
        x = self.fc5(x)
        x = self.rrelu(x)
        x = self.fc6(x)
        x = self.relu(x)
        
        return x

In [ ]:
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
best_net = None
epochs = [100, 101]
depths = [2, 3,]
_mae = 20 ** 10

In [ ]:
for depth in depths:
    for _epochs in epochs:
        for train_index, test_index in kf.split(data_train.drop(target_column, axis=1), data_train[target_column]):
            train_data, test_data = data.iloc[train_index], data.iloc[test_index]

            train_dataset = CustomDataset(train_data, train=True)
            train_loader = DataLoader(train_dataset, batch_size=500, shuffle=True)

            test_dataset = CustomDataset(test_data, train=True)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

            model = Net(data.shape[1] - 1, depth).to(DEVICE)
            criterion = nn.MSELoss().to(DEVICE)
            optimizer = torch.optim.AdamW(model.parameters())

            for epoch in range(_epochs):
                train_model(model, train_loader, optimizer, criterion)
            
            model.eval()
            all_preds = []
            all_targets = []
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    all_preds.append(outputs.detach().cpu().numpy()[0])
                    all_targets.append(labels.detach().cpu().numpy()[0])
            mae = mean_absolute_error(all_targets, all_preds)
            print(f"MAE: {mae}, epochs: {_epochs}, depth: {depth}")

            if mae < _mae:
                _mae = mae
                best_net = model

In [ ]:
_mae

In [ ]:
importances = catboost_model.feature_importances_
imp = pd.DataFrame(data_test.drop(target_column, axis=1).columns, columns=["Feature"])
imp["Importance"] = importances

imp = imp.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(imp['Feature'], imp['Importance'])
plt.xlabel('Признаки')
plt.ylabel('Важность')
plt.title('График важности признаков')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
predictions = forest_model.predict(data_test.drop(target_column, axis=1))
mae = mean_absolute_error(data_test[target_column], predictions)
r2 = r2_score(data_test[target_column], predictions)
rmse = mean_squared_error(data_test[target_column], predictions, squared=False)
add_table(("RandomForest_test", mae, r2, rmse))

In [ ]:
forest_model